# 7. Optional - Advanced Mapping Techniques


- [7.1 Introduction](#section1)
- [7.2 More Mapping Points](#sectio2) 
    - Graduated Symbol Maps
- [7.3 Saving your maps](#section3)
- [7.4 Bivariate Mapping](#section4)
- [7.5 Faceted Mapping](#section5)

**INSTRUCTOR NOTES**:
- Datasets used:
    - "../notebook_data/outdata/tracts_acs_gdf_ac.json"
    - "../notebook_data/outdata/tracts_acs_gdf_sf.json"
- Expected time to complete:
    - 1.5 hours

<a id="section1"></a>
## 7.1 Introduction


In this optional notebook you'll learn another point mapping technique and how to combine point and choropleth maps to create a bivariate map. We then discuss faceted mapping to look at variables and how they evolve over time.

## Set-up

First confirm that you have the following packages installed. 
Let's import the packages as we did with our last notebook.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import geojson # ditto for GeoJSON data - an extension of JSON with support for geographic data

import matplotlib # base python plotting library
import matplotlib.pyplot as plt # submodule of matplotlib

import mapclassify # to classify data values

# To display plots, maps, charts etc in the notebook
%matplotlib inline  

<a id="section 2"></a>
## 7.2 More Mapping Points
### Graduated symbol mapping

We'll now move on to **graduated symbol mapping**. Proportional Symbol maps are unclassed, and the size of our markers are for absolute magnitude. With graduated symbols, we use a classification scheme to set the size of point symbols.

Let's map the count of renters (`c_renters`) again. We can first use `.describe()`, which we used before, to see what values are at the 25th, 50th, and 75th percentiles.

In [ ]:
# Import tracts data
tracts_acs_gdf = gpd.read_file("../notebook_data/outdata/tracts_acs_gdf_ac.json")

In [ ]:
# Create point dataset
tracts_acs_gdf_point = gpd.GeoDataFrame(tracts_acs_gdf.loc[:,tracts_acs_gdf.columns!='geometry'], 
                            geometry=tracts_acs_gdf.centroid)

In [ ]:
tracts_acs_gdf_point['c_renters'].describe()

Unfortunately, we cannot use the plot `scheme` argument to classify the data being mapped.  But we can manually create a classification for the data using the underlying tools.

In [ ]:
quantile_bins = mapclassify.Quantiles(tracts_acs_gdf_point['c_renters'], k=5)
quantile_bins

Above, we created five bins (`k=5`) from the data in the `c_renters` column. When we display `quantile_bins` we see the bin ranges and the number of observations in each bin.

We can get the number of bins out of the `quantile_bins` object:

In [ ]:
# Number of bins
quantile_bins.k

The `yb` attribute of `quantile_bins` will give us the bin identifier for each `c_renters` value.

In [ ]:
print(quantile_bins.yb)

print("\nDo we have a bin id for each tract?", len(quantile_bins.yb) == len(tracts_acs_gdf_point['c_renters']))

The `bins` attribute will give us the upper bounds of each bin.

In [ ]:
quantile_bins.bins

We can use these bins to create custom labels for our graduated symbol map legend.

In [ ]:
renter_label = ["< 293", "293 - 542", "542 - 788", "788 - 1142", "> 1142"]

We can then loop through our quantile groups, plot them in different sizes, and create a legend.

In [ ]:
fig, ax = plt.subplots(figsize = (15,15)) 

# Add the census tract polygons in the background
tracts_acs_gdf.plot(color='lightgrey',
                    edgecolor='white',
                    ax=ax)

# Overlay the points - setting the size based on the quantile bin
for i in range(quantile_bins.k):
    # add the tract point data as separate layers - one four each quantile bin
    tracts_acs_gdf_point[quantile_bins.yb==i].plot(
                                         edgecolor='red',
                                         color="pink",                             
                                         alpha=0.4, 
                                         linewidths=1, 
                                         markersize=(3**(i+1)),  #The number three to the power of i+1
                                         legend=True,
                                         label=renter_label[i],
                                         ax=ax)
                                         

plt.legend()
plt.show()

#### Exercise

Redo the above with `FisherJenks` classification. Recall that `FisherJenks` minimizes within-class variance and maximize between-class differences. This is great for exploratory data analysis since it can identify natural groupings in the data. 

In [ ]:
# Your code here

*Click here for answers*

<!--- 
    # SOLUTION
    jenks_bins= mapclassify.FisherJenks(tracts_acs_gdf_point['c_renters'], k=5)
    jenks_bins

    # SOLUTION
    renter_labels_jenks = ["< 436","436 - 839","839 - 1211","1211 - 1781", "> 1781"]

    fig, ax = plt.subplots(figsize = (15,15)) 

    tracts_acs_gdf.plot(color='lightgrey',
                        edgecolor='white',
                        ax=ax)

    for i in range(jenks_bins.k):
        tracts_acs_gdf_point[jenks_bins.yb==i].plot(
                                             edgecolor='darkorchid',
                                             color="mediumpurple",                             
                                             alpha=0.4, 
                                             linewidths=1, 
                                             markersize=(3**(i+1)),  #The number three to the power of i+1
                                             label=renter_labels_jenks[i],
                                             ax=ax)

    plt.legend()
    plt.tight_layout()
    plt.title("Alameda County, Count of Renters by Census Tract")
    plt.show()
--->

<a id="section3"></a>
## 7.3 Saving your maps

You can save your maps with the `plt.savefig` method.

> **Important**: you must `plt.savefig` your map before you display it with `plt.show()`! 

For example...

In [ ]:
fig, ax = plt.subplots(figsize = (15,15)) 

# Add the census tract polygons in the background
tracts_acs_gdf.plot(color='lightgrey',
                    edgecolor='white',
                    ax=ax)

# Overlay the points - setting the size based on the quantile bin
for i in range(quantile_bins.k):
    # add the tract point data as separate layers - one four each quantile bin
    tracts_acs_gdf_point[quantile_bins.yb==i].plot(
                                         edgecolor='red',
                                         color="pink",                             
                                         alpha=0.4, 
                                         linewidths=1, 
                                         markersize=(3**(i+1)),  #The number three to the power of i+1
                                         legend=True,
                                         label=renter_label[i],
                                         ax=ax)
                                         

plt.legend()
plt.tight_layout()
plt.title("Alameda County, Count of Renters by Census Tract")
 
# Save figure before showing it
plt.savefig('./img/AC_renters.png')

# Display map in the notebook
plt.show()

In [ ]:
### Take a look at your saved map file 

#### You can view your saved file by uncommenting the line below and then changing this code cell to markdown.
#### you can also browse to the folder in which it resides and click on it to view

#<img src = "./img/AC_renters.png">

<a id="section4"></a>
## 7.4 Bivariate Mapping

**Bivariate mapping** is a technique that involves mapping 2 different variables to visually explore any spatial relationship between the variables of interest. 

Let's compare and investigate the correlation between the non-white population percentages and median household income.

Remember how we did a bivariate plot in part 1? Let's remake that plot here.

In [ ]:
# First create a percent non-white variable
tracts_acs_gdf['pct_nonwhite'] = 100 * ((tracts_acs_gdf['c_race'] - tracts_acs_gdf['c_white']) / tracts_acs_gdf['c_race'])

Now, create a non-spatial bivariate graph just to see what the relationship might be between the two variables if we're not thinking about them spatially.

In [ ]:
# Bivariate
# plot it
plt.scatter(tracts_acs_gdf['pct_nonwhite'],
            tracts_acs_gdf['med_hhinc'])
plt.xlabel('Percent Non-White')
plt.ylabel('Est. Median Household Income')

We can see from the above that there is an inverse relationship between these two variables.

Our goal here is to take this idea, and expand it to visualize it spatially. Here we'll use the variables `pct_nonwhite` and `med_hhinc`.

We can visualize both variables spatially if we map one geodataframe as a `choropleth map layer` and one as a `graduated color map layer`.


In [ ]:
fig, ax = plt.subplots(figsize = (15,8)) 

# Plot percent non-white as choropleth
tracts_acs_gdf.plot(column='pct_nonwhite', 
                        legend=True,
                        legend_kwds={'title': 'Percent Non-white', 'loc':'upper right'},
                        cmap="Blues", 
                        scheme='user_defined', 
                        classification_kwds={'bins':[20, 40, 60, 80]}, 
                        ax=ax)

# Plot median household income as graduated color symbols
tracts_acs_gdf_point.plot(
    column=tracts_acs_gdf_point['med_hhinc'],
    cmap='Reds',
    alpha=1, 
    linewidth=0.5, 
    markersize=60,
    legend=True,
    legend_kwds={'label': "Median Household Income ($)"},
    ax=ax)

plt.title("Alameda County, Percent Non-White / Median Household Income")
plt.tight_layout()
plt.show()

To customize this a couple steps further, you can do things like title our `pct_nonwhite` legend and remove our tick marks. 

In [ ]:
fig, ax = plt.subplots(figsize = (15,6)) 

# Plot percent non-white as choropleth
pct_nonwhite_choro = tracts_acs_gdf.plot(column='pct_nonwhite', 
                                       cmap="Blues", 
                                       legend=True,
                                       scheme='user_defined', 
                                       classification_kwds={'bins':[20, 40,60,80]},
                                       legend_kwds={'loc':'upper right','title':"Percent non-white\n"},
                                       ax=ax)
# Customize the legend labels
legend_labels_list = ['<20%','20% - 40%','40% - 60%','60% - 80%','80% - 100%']
for j in range(0,len(ax.get_legend().get_texts())):
        ax.get_legend().get_texts()[j].set_text(legend_labels_list[j])

# Plot median household income as points with color based on rent amount
tracts_acs_gdf_point.plot(
    cmap='Reds',
    column=tracts_acs_gdf_point['med_hhinc'],
    alpha=0.8, 
    linewidth=0.5, 
    markersize=50,
    legend=True,
    legend_kwds={'label': "Median Household Income ($)"},
    ax=ax)

# Remove tickmarks
plt.tick_params(axis='both',  # both x and y axis
                which='both', # both major and minor
                bottom=False, # don't show bottom ticks
                left=False)   # don't show left ticks

plt.tight_layout()
plt.show()

Beautiful! So that we can put this in whatever document, publication, website etc. we'll try saving it out.

In [ ]:
fig, ax = plt.subplots(figsize = (15,6)) 

# Plot percent non-white as choropleth
pct_nonwhite_choro = tracts_acs_gdf.plot(column='pct_nonwhite', 
                                       cmap="Blues", 
                                       legend=True,
                                       scheme='user_defined', 
                                       classification_kwds={'bins':[20, 40,60,80]},
                                       legend_kwds={'loc':'upper right','title':"Percent non-white\n"},
                                       ax=ax)
legend_labels_list = ['<20%','20% - 40%','40% - 60%','60% - 80%','80% - 100%']
for j in range(0,len(ax.get_legend().get_texts())):
        ax.get_legend().get_texts()[j].set_text(legend_labels_list[j])

# Plot median rent as points with color based on rent amount
tracts_acs_gdf_point.plot(
    cmap='Reds',
    column=tracts_acs_gdf_point['med_rent'],
    alpha=0.8, 
    linewidth=0.5, 
    markersize=50,
    legend=True,
    legend_kwds={'label': "Median Rent ($)"},
    ax=ax)

# For removing tickmarks
plt.tick_params(axis='both',  # both x and y axis
                which='both', # both major and minor
                bottom=False, # don't show bottom ticks
                left=False)   # don't show left ticks

plt.tight_layout()


# Save figure before showing it
plt.savefig('../outdata/AC_nonwhite_med_hhinc.png')

plt.show()

#### Exercise
Make a bivariate map for SF!
Then discuss a partner about the effectiveness of the map you made and determine what you could do differently.

In [ ]:
# Your code here

*Click here for answers*

<!--- 
    # SOLUTION
    tracts_acs_gdf_sf = gpd.read_file("../notebook_data/outdata/tracts_acs_gdf_sf.json")
    tracts_acs_gdf_sf['pct_nonwhite'] = 100 * ((tracts_acs_gdf_sf['c_race'] - tracts_acs_gdf_sf['c_white']) / tracts_acs_gdf_sf['c_race'])
    tracts_acs_gdf_sf.pct_nonwhite

    # SOLUTION
    tracts_acs_gdf_sf_point = gpd.GeoDataFrame(tracts_acs_gdf_sf.loc[:,tracts_acs_gdf_sf.columns!='geometry'], 
                                geometry=tracts_acs_gdf_sf.centroid)

    # SOLUTION

    fig, ax = plt.subplots(figsize = (15,6)) 

    # Plot percent non-white as choropleth
    tracts_acs_gdf_sf.plot(column='pct_nonwhite', 
                            legend=True, 
                            label="%Non-white",
                            cmap="Blues", 
                            scheme='user_defined', 
                            classification_kwds={'bins':[20, 40,60,80]}, 
                            ax=ax)

    # Plot median rent as points with color based on rent amount
    tracts_acs_gdf_sf_point.plot(
        cmap='Reds',
        column=tracts_acs_gdf_sf_point['med_rent'],
        alpha=0.8, 
        linewidth=0.5, 
        markersize=50,
        legend=True,
        legend_kwds={'label': "Median Rent ($)"},
        ax=ax)

    plt.tight_layout()
    plt.show()
--->

<a id="section5"></a>
## 7.5 Faceted mapping

**Faceted mapping** is where we arrange numerous maps side-by-side to see how variables such as time effect our maps. Each map within this series usually has the shape extent and shape. We can also use this method to see how different variables have diverging or related spatial patterns. 

In order to do a temporal comparison, let's pull in our 2013 tract and census data. We'll go through the same process as we did in s3_1 to prepare it.

In [ ]:
# Import 2013 tract data
tracts_gdf_2013 = gpd.read_file('zip://../notebook_data/census/Tracts/cb_2013_06_tract_500k.zip')
tracts_gdf_2013 = tracts_gdf_2013[tracts_gdf_2013['COUNTYFP']=='001']
tracts_gdf_2013.head(2)

In [ ]:
# Import ACS data
acs5data_df_2013 = pd.read_csv("../notebook_data/census/ACS5yr/census_variables_CA_2013.zip", dtype={'FIPS_11_digit':str})
acs5data_df_2013.head(2)

In [ ]:
# Create a percent non-white variable
acs5data_df_2013['pct_nonwhite'] = 100 * ((acs5data_df_2013['c_race'] - acs5data_df_2013['c_white']) / acs5data_df_2013['c_race'])

In [ ]:
# Fix FIPS code
acs5data_df_2013['FIPS_11_digit']

In [ ]:
acs5data_df_2013['FIPS_11_digit']

In [ ]:
# Inner join keeps only data where both tract geometry and demographic data exist for the tract
tracts_acs_gdf_2013 = tracts_gdf_2013.merge(acs5data_df_2013, left_on='GEOID',right_on="FIPS_11_digit", how='inner')
tracts_acs_gdf_2013.head()

Let's check to make sure we have the variables we want to compare

In [ ]:
tracts_acs_gdf_2013[['pct_nonwhite', 'med_rent', 'med_hhinc']]

We'll save these out as the variables we want to compare so we can refer to them later.

In [ ]:
mapvars = ['pct_nonwhite', 'med_rent', 'med_hhinc']

First, let's map each variable separately. 

In [ ]:
# Choropleth map of Pct Nonwhite
tracts_acs_gdf_2013[tracts_acs_gdf_2013['pct_nonwhite']>0].plot(column = 'pct_nonwhite',
                                                              scheme='user_defined',
                                                              classification_kwds={'bins':[20,40,60,80]}, 
                                                              legend=True, 
                                                              legend_kwds={'loc': 'upper left'})
plt.show()

In [ ]:
# Choropleth map of Median household rent
tracts_acs_gdf_2013[tracts_acs_gdf_2013['med_rent']>0].plot(column = 'med_rent',
                                                              scheme='user_defined',
                                                              classification_kwds={'bins':[500,1000,1500,2000,4000]}, 
                                                              legend=True, 
                                                              legend_kwds={'loc': 'upper left'})
plt.show()

Now it's your turn! Make a choropleth for `med_hhinc`.

In [ ]:
# Choropleth map of median household income

*Click here for answers*

<!---
    tracts_acs_gdf_2013[tracts_acs_gdf_2013['med_hhinc']>0].plot(column = 'med_hhinc',
                                                                  scheme='user_defined',
                                                                  classification_kwds={'bins':[50000,75000,100000,150000]}, 
                                                                  legend=True, 
                                                                  legend_kwds={'loc': 'upper left'})
    plt.show()
--->

#### Extra Challenge 
Try making all of these maps using a `for` loop!

In [ ]:
# Your code here

*Click here for answers*

<!--- 
    mapvars = ['pct_nonwhite', 'med_rent', 'med_hhinc']
    break_dict = {'pct_nonwhite':[20,40,60,80],
                  'med_rent':[500,1000,1500,2000,4000],
                  'med_hhinc':[50000,75000,100000,150000]}

    for mapvar in mapvars:
        tracts_acs_gdf_2013[tracts_acs_gdf_2013[mapvar]>0].plot(column = mapvar,
                                                                  scheme='user_defined',
                                                                  classification_kwds={'bins':break_dict[mapvar]}, 
                                                                  legend=True, 
                                                                  legend_kwds={'loc': 'upper left'})
        plt.show()
--->

We're now going to start making our facet map.

To be able to loop through our variables, we're first going to make a couple of dictionaries. Each dictionary holds the following information:
- `break_dict` -  Thresholds for each group within the variable
- `legend_labels_dict` - Legend labels according to thresholds of each group
- `title_dict` - Titles for each subplot according to the variable

In [ ]:
break_dict = {'pct_nonwhite':[20,40,60,80],
              'med_rent':[500,1000,1500,2000,4000],
              'med_hhinc':[50000,75000,100000,150000]}

legend_labels_dict = {
              'pct_nonwhite':['<20%','20% - 40%','40% - 60%','60% - 80%','80% - 100%'], 
              'med_rent':["<$500",'\$500 - $1,000','\$1,000 - $1,500','\$1,500 - $2,000','\$2,000 - $4,000+'],
              'med_hhinc':['<$50,000','\$50,000 - $75,000','\$75,000 - $100,000','\$100,000 - $150,000','>$150,000']}
    
title_dict = {'pct_nonwhite':'Pct Non-white or Asian',
              'med_rent': 'Median Rent',
              'med_hhinc':'Median Household Income'}

We'll focus on 2013 first. Walk through this code step by step and discuss with a neighbor what you think is happening here.

In [ ]:
f, axs = plt.subplots(nrows=1, ncols=3, figsize=(18,16))
# Make the axes accessible with single indexing
axs = axs.flatten()

# Start a loop over all the variables of interest
for i, col in enumerate(mapvars):
    # select the axis where the map will go
    ax = axs[i]
    # Plot the map
    tracts_acs_gdf_2013[tracts_acs_gdf_2013[col]>0].plot(col,
                                                 scheme='user_defined', 
                                                 classification_kwds={'bins':break_dict[col]}, 
                                                 ax=ax, 
                                                 legend=True, legend_kwds={'loc': 'upper left'})
    # Remove axis clutter
    ax.set_axis_off()
    # Set the axis title to the name of variable being plotted
    ax.set_title(title_dict[col]+ ', 2013')
    # fix the legend labels
    for j in range(0,len(ax.get_legend().get_texts())):
        ax.get_legend().get_texts()[j].set_text(legend_labels_dict[col][j])
# Display the figure
plt.show()

On to our 2018 data!

In [ ]:
break_dict = {'pct_nonwhite':[20,40,60,80],
              'med_rent':[500,1000,1500,2000,4000],
              'med_hhinc':[50000,75000,100000,150000]}

legend_labels_dict = {
              'pct_nonwhite':['0% - 20%','20% - 40%','40% - 60%','60% - 80%','80% - 100%'], 
              'med_rent':["\$0 - $500",'\$500 - $1,000','\$1,000 - $1,500','\$1,500 - $2,000','\$2,000 - $4,000+'],
              'med_hhinc':['\$0 - $50,000','\$50,000 - $75,000','\$75,000 - $100,000','\$100,000 - $150,000','>$150,000']}
    
title_dict = {'pct_nonwhite':'Pct Non-white or Asian',
              'med_rent': 'Median Rent',
              'med_hhinc':'Median Household Income'}

f, axs = plt.subplots(nrows=1, ncols=3, figsize=(18,16))

# Make the axes accessible with single indexing
axs = axs.flatten()
# Start a loop over all the variables of interest
for i, col in enumerate(mapvars):
    # select the axis where the map will go
    ax = axs[i]
    # Plot the map
    #db.plot(column=col, ax=ax, scheme='Quantiles', 
    #        linewidth=0, cmap='RdPu')
    tracts_acs_gdf[tracts_acs_gdf[col]>0].plot(col,
                                       scheme='user_defined', 
                                       classification_kwds={'bins':break_dict[col]},
                                       ax=ax, 
                                       legend=True, legend_kwds={'loc': 'upper left'})
    
    #ax.get_legend().set_text(legend_labels_dic[col])
    # Remove axis clutter
    ax.set_axis_off()
    # Set the axis title to the name of variable being plotted
    #ax.set_title(col)
    ax.set_title(title_dict[col]+ ', 2018')
    # fix the legend labels
    for j in range(0,len(ax.get_legend().get_texts())):
        ax.get_legend().get_texts()[j].set_text(legend_labels_dict[col][j])

# Display the figure
plt.show()

Amazing, time to put it all together!

In [ ]:
f, axs = plt.subplots(nrows=2, ncols=3, figsize=(18,10))
# Make the axes accessible with single indexing
axs = axs.flatten()

# Start a loop over all the variables of interest
for i, col in enumerate(mapvars):
    # select the axis where the map will go
    ax = axs[i]
    # Plot the map
    tracts_acs_gdf_2013[tracts_acs_gdf_2013[col]>0].plot(col,
                                                 scheme='user_defined', 
                                                 classification_kwds={'bins':break_dict[col]}, 
                                                 ax=ax, 
                                                 legend=True, legend_kwds={'loc': 'lower left'})
    # Remove axis clutter
    ax.set_axis_off()
    # Set the axis title to the name of variable being plotted
    ax.set_title(title_dict[col]+ ', 2013')
    # fix the legend labels
    for j in range(0,len(ax.get_legend().get_texts())):
        ax.get_legend().get_texts()[j].set_text(legend_labels_dict[col][j])

# Start a loop over all the variables of interest
for i, col in enumerate(mapvars):
    # select the axis where the map will go
    ax = axs[i+3]
    # Plot the mapaa
    tracts_acs_gdf[tracts_acs_gdf[col]>0].plot(col,
                                       scheme='user_defined', 
                                       classification_kwds={'bins':break_dict[col]},
                                       ax=ax, 
                                       legend=True, legend_kwds={'loc': 'lower left'})
    
    #ax.get_legend().set_text(legend_labels_dic[col])
    # Remove axis clutter
    ax.set_axis_off()
    # Set the axis title to the name of variable being plotted
    #ax.set_title(col)
    ax.set_title(title_dict[col]+ ', 2018')
    # fix the legend labels
    for j in range(0,len(ax.get_legend().get_texts())):
        ax.get_legend().get_texts()[j].set_text(legend_labels_dict[col][j])

plt.tight_layout()
# Display the figure
plt.show()

Pat yourself on the back! You made your first facet map. Discuss with your partner how you could make this better.

#### Exercise

Make your own faceted map for SF. You'll need to do the following steps
1. Prepare SF 2013 tracts data
2. Join with the ACS data
3. Step-by-step create your facet map with variables of your choosing!

In [ ]:
# Your code here

*Click here for answers*

<!--- 
    # SOLUTION
    tracts_gdf_sf_2013 = gpd.read_file('zip://../notebook_data/census/Tracts/cb_2013_06_tract_500k.zip')
    tracts_gdf_sf_2013 = tracts_gdf_sf_2013[tracts_gdf_sf_2013['COUNTYFP']=='075']
    tracts_gdf_sf_2013.head(2)

    # SOLUTION
    # Inner join keeps only data where both tract geometry and demographic data exist for the tract
    tracts_acs_gdf_sf_2013 = tracts_gdf_sf_2013.merge(acs5data_df_2013, left_on='GEOID',right_on="FIPS_11_digit", how='inner')
    tracts_acs_gdf_sf_2013.head()


    # SOLUTION

    break_dict = {'pct_nonwhite':[20,40,60,80],
                  'med_rent':[500,1000,1500,2000,4000],
                  'med_hhinc':[50000,75000,100000,150000]}
    #legend_labels = ['< 2.4', '2.4 - 6', '6 - 15', '15 - 38', '38 - 140 M']
    legend_labels_dict = {
                  'pct_nonwhite':['0% - 20%','20% - 40%','40% - 60%','60% - 80%','80% - 100%'], 
                  'med_rent':["\$0 - $500",'\$500 - $1,000','\$1,000 - $1,500','\$1,500 - $2,000','\$2,000 - $4,000+'],
                  'med_hhinc':['\$0 - $50,000','\$50,000 - $75,000','\$75,000 - $100,000','\$100,000 - $150,000','>$150,000']}

    title_dict = {'pct_nonwhite':'Pct Non-white or Asian',
                  'med_rent': 'Median Rent',
                  'med_hhinc':'Median Household Income'}

    f, axs = plt.subplots(nrows=2, ncols=3, figsize=(18,10))
    # Make the axes accessible with single indexing
    axs = axs.flatten()

    # Start a loop over all the variables of interest
    for i, col in enumerate(mapvars):
        # select the axis where the map will go
        ax = axs[i]
        # Plot the map
        tracts_acs_gdf_sf_2013[tracts_acs_gdf_sf_2013[col]>0].plot(col,
                                                     scheme='user_defined', 
                                                     classification_kwds={'bins':break_dict[col]}, 
                                                     ax=ax, 
                                                     legend=True, legend_kwds={'loc': 'upper left'})
        # Remove axis clutter
        ax.set_axis_off()
        # Set the axis title to the name of variable being plotted
        ax.set_title(title_dict[col]+ ', 2013')
        # fix the legend labels
        for j in range(0,len(ax.get_legend().get_texts())):
            ax.get_legend().get_texts()[j].set_text(legend_labels_dict[col][j])


    break_dict = {'pct_nonwhite':[20,40,60,80],
                  'med_rent':[500,1000,1500,2000,4000],
                  'med_hhinc':[50000,75000,100000,150000]}
    #legend_labels = ['< 2.4', '2.4 - 6', '6 - 15', '15 - 38', '38 - 140 M']
    legend_labels_dict = {
                  'pct_nonwhite':['0% - 20%','20% - 40%','40% - 60%','60% - 80%','80% - 100%'], 
                  'med_rent':["\$0 - $500",'\$500 - $1,000','\$1,000 - $1,500','\$1,500 - $2,000','\$2,000 - $4,000+'],
                  'med_hhinc':['\$0 - $50,000','\$50,000 - $75,000','\$75,000 - $100,000','\$100,000 - $150,000','>$150,000']}

    title_dict = {'pct_nonwhite':'Pct Non-white or Asian',
                  'med_rent': 'Median Rent',
                  'med_hhinc':'Median Household Income'}

    # Start a loop over all the variables of interest
    for i, col in enumerate(mapvars):
        # select the axis where the map will go
        ax = axs[i+3]
        # Plot the mapaa
        tracts_acs_gdf_sf[tracts_acs_gdf_sf[col]>0].plot(col,
                                           scheme='user_defined', 
                                           classification_kwds={'bins':break_dict[col]},
                                           ax=ax, 
                                           legend=True, legend_kwds={'loc': 'upper left'})

        #ax.get_legend().set_text(legend_labels_dic[col])
        # Remove axis clutter
        ax.set_axis_off()
        # Set the axis title to the name of variable being plotted
        #ax.set_title(col)
        ax.set_title(title_dict[col]+ ', 2018')
        # fix the legend labels
        for j in range(0,len(ax.get_legend().get_texts())):
            ax.get_legend().get_texts()[j].set_text(legend_labels_dict[col][j])

    plt.tight_layout()
    # Display the figure
    plt.show()
--->

## 7.6 Recap
In this workbook we learned about how to classify and bin, as well as create three new types of maps: graduated symbol, bivariate, and faceted. 

Below is a recap of the concepts and methods we covered:

- Classifying and binning
	- `mapclassify.Quantiles()`
	- `.bins`
- Different types of maps
    - Graduated Symbol
    - Bivariate
    - Faceted
 



## Congrats you're done with part s3_7!


---
<div style="display:inline-block;vertical-align:middle;">
<a href="https://dataforhousing.org/" target="_blank"><img src ="https://media-exp1.licdn.com/dms/image/C560BAQELkt35AxeIeA/company-logo_200_200/0?e=1597881600&v=beta&t=irZ1tYCA9A2biVzCguvCXzsfzanSYDFuF22IUFNY5Sg" width="75" align="left">
</a>
</div>

<div style="display:inline-block;vertical-align:middle;">
    <div style="font-size:larger">&nbsp;Data Science for Housing Workshop, University of California Berkeley</div>
    <div>&nbsp;Tim Thomas, Patty Frontiera, Emmanuel Lopez, Ethan Ebinger, Hikari Murayama, Karen Chapple, Claudia von Vacano<div>
    <div>&copy; UC Regents, 2019-2020</div>
</div>